In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# load model
model = tf.keras.models.load_model('/content/gdrive/My Drive/data/stereo_matching/model_cosine_fit_v1.h5')
#model.summary()

In [3]:
left  = cv2.imread("/content/gdrive/My Drive/data/stereo_matching/Aloe_left_1.png", 0)
right = cv2.imread("/content/gdrive/My Drive/data/stereo_matching/Aloe_right_2.png", 0)

#z-score
mean = np.mean(left)
std  = np.std(left)
left = (left - mean)/(std+1e-7)

mean = np.mean(right)
std  = np.std(right)
right = (right - mean)/(std+1e-7)

disparity_range = 64
H, W = right.shape

print(left.shape)
print(right.shape)
print(H, W)

(370, 427)
(370, 427)
370 427


In [4]:
cost_volume = np.zeros((356, 413, disparity_range))
for loc in range(disparity_range):
    x_off = -loc
    l = left[:, max(0, -x_off):W]
    r = right[:, 0:min(W, W + x_off)]
    
    lr = tf.stack([l,r], axis=2)
    lr = tf.expand_dims(lr, axis=0)
    
    similarity_map = tf.squeeze(model.predict(lr))
    #cost_volume[:, 0:similarity_map.shape[2], loc] = similarity_map[:, :]
    cost_volume[:, loc:413, loc] = similarity_map[:, :]

ValueError: ignored

In [ ]:
from PIL import Image

pred = tf.argmax(cost_volume, axis=2)
pred = pred.numpy() * 3.0

# save
pred = pred.astype(np.uint8)
im = Image.fromarray(pred)
im.save('/content/gdrive/My Drive/data/stereo_matching/model_cosine_fit_v1.png')